# SK-5-VectorStores : RAG avec Qdrant

**Navigation** : [<< 04-Filters](04-SemanticKernel-Filters-Observability.ipynb) | [Index](README.md) | [06-ProcessFramework >>](06-SemanticKernel-ProcessFramework.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre l'architecture **Vector Store** de SK
2. Generer des **embeddings** avec OpenAI
3. Utiliser **InMemoryVectorStore** pour le prototypage
4. Connecter **Qdrant** pour la production
5. Implementer un pipeline **RAG** complet

### Prerequis

- Python 3.10+
- Notebooks 01-04 completes
- Cle API OpenAI configuree (`.env`)
- Acces Qdrant (optionnel, fourni)

### Duree estimee : 50 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Introduction | Pourquoi les Vector Stores ? |
| 2 | Architecture SK | VectorStore, Collections, Records |
| 3 | Embeddings | OpenAITextEmbedding |
| 4 | InMemoryVectorStore | Prototypage rapide |
| 5 | Qdrant | Production-ready |
| 6 | RAG Pattern | Pipeline complet |
| 7 | Conclusion | Resume, exercices |

> **Qu'est-ce qu'un Vector Store ?** Une base de donnees optimisee pour stocker et rechercher des vecteurs (embeddings). C'est la fondation du RAG (Retrieval-Augmented Generation) qui permet aux LLMs d'acceder a vos donnees.

In [1]:
# Installation
%pip install semantic-kernel qdrant-client python-dotenv --quiet

import os
from dotenv import load_dotenv

# Chargement du fichier .env (cles API)
load_dotenv("../.env")

api_key = os.getenv("OPENAI_API_KEY")
print(f"Configuration: API Key {'OK' if api_key else 'MANQUANTE'}")
print("Dependances installees")

Note: you may need to restart the kernel to use updated packages.
Configuration: API Key OK
Dependances installees



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\jsboi\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


## 1. Introduction aux Vector Stores

### Pourquoi les Vector Stores ?

Les LLMs ont une limite de contexte et pas d'acces a vos donnees privees. Les Vector Stores resolvent ce probleme :

```
Documents          Embeddings          Vector Store
┌─────────┐       ┌─────────┐         ┌─────────────┐
│ Doc 1   │──────>│ [0.1,   │────────>│             │
│         │       │  0.3,   │         │   Qdrant    │
└─────────┘       │  ...]   │         │             │
                  └─────────┘         │  ou autre   │
                                      └─────────────┘
                                            |
Query: "Qu'est-ce que X ?"                  |
        |                                   |
        v                                   v
   Embedding Query ──────> Recherche similitude
        |                                   |
        v                                   v
   [0.2, 0.4, ...]         Top-K documents pertinents
                                    |
                                    v
                           Contexte pour le LLM
```

### Connecteurs SK disponibles

| Connecteur | Type | Cas d'usage |
|------------|------|-------------|
| **InMemoryVectorStore** | Local | Prototypage, tests |
| **QdrantVectorStore** | Cloud/Self-hosted | Production |
| **AzureAISearchVectorStore** | Azure | Enterprise |
| **PineconeVectorStore** | Cloud | Scalabilite |
| **RedisVectorStore** | Cache distribue | Performance |

## 2. Architecture Vector Store SK

SK utilise une abstraction a trois niveaux :

```
┌─────────────────────────────────────────────┐
│              VectorStore                    │
│  (InMemory, Qdrant, Azure, Pinecone, ...)   │
│                                             │
│   ┌─────────────────────────────────────┐  │
│   │    VectorStoreRecordCollection      │  │
│   │    (equivalent d'une "table")       │  │
│   │                                     │  │
│   │   ┌─────────────────────────────┐  │  │
│   │   │  VectorStoreRecordDefinition│  │  │
│   │   │  (schema des records)       │  │  │
│   │   └─────────────────────────────┘  │  │
│   └─────────────────────────────────────┘  │
└─────────────────────────────────────────────┘
```

### Concepts cles

| Concept | Description | Analogie SQL |
|---------|-------------|-------------|
| **VectorStore** | Connexion a la base | Database connection |
| **Collection** | Groupe de records | Table |
| **Record** | Document + embedding | Row |
| **Key** | Identifiant unique | Primary Key |
| **Vector** | Embedding du contenu | Colonne indexee |

## 3. Generation d'Embeddings

In [2]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAITextEmbedding

# Configuration
kernel = Kernel()

# Service d'embedding
embedding_service = OpenAITextEmbedding(
    service_id="embedding",
    ai_model_id="text-embedding-3-small"  # Modele recommande (peu couteux, performant)
)
kernel.add_service(embedding_service)

# Test de generation d'embedding
test_texts = [
    "Semantic Kernel est un SDK pour l'IA",
    "Python est un langage de programmation"
]

embeddings = await embedding_service.generate_embeddings(test_texts)

print(f"Nombre de textes: {len(test_texts)}")
print(f"Dimension des embeddings: {len(embeddings[0])}")
print(f"Premier embedding (debut): {embeddings[0][:5]}...")

Nombre de textes: 2
Dimension des embeddings: 1536
Premier embedding (debut): [-0.03927445  0.0422161  -0.02658716 -0.0338627   0.00596752]...


### Interpretation : Embeddings

Les embeddings sont des representations vectorielles du sens :

| Propriete | Valeur | Signification |
|-----------|--------|---------------|
| **Dimension** | 1536 (text-embedding-3-small) | Complexite de la representation |
| **Plage** | [-1, 1] | Valeurs normalisees |
| **Similarite** | Cosinus ou dot product | Plus proche = plus similaire |

**Modeles OpenAI disponibles** :

| Modele | Dimension | Prix | Usage |
|--------|-----------|------|-------|
| `text-embedding-3-small` | 1536 | $0.02/1M tokens | Recommande |
| `text-embedding-3-large` | 3072 | $0.13/1M tokens | Haute precision |
| `text-embedding-ada-002` | 1536 | $0.10/1M tokens | Legacy |

## 4. InMemoryVectorStore

Pour le prototypage rapide sans infrastructure externe.

In [3]:
from dataclasses import dataclass
from typing import Annotated
from semantic_kernel.connectors.in_memory import InMemoryStore
from semantic_kernel.data.vector import (
    vectorstoremodel,
    VectorStoreField,
    FieldTypes
)

# Definition du schema de record avec la nouvelle API SK 1.39+
@vectorstoremodel
@dataclass
class DocumentRecord:
    """Schema d'un document dans le vector store."""
    id: Annotated[str, VectorStoreField(field_type=FieldTypes.KEY)]
    content: Annotated[str, VectorStoreField(field_type=FieldTypes.DATA)]
    title: Annotated[str, VectorStoreField(field_type=FieldTypes.DATA)]
    embedding: Annotated[
        list[float] | None,
        VectorStoreField(
            field_type=FieldTypes.VECTOR,
            dimensions=1536
        )
    ] = None

# Creation du store en memoire
memory_store = InMemoryStore()

# Obtenir ou creer une collection
collection = memory_store.get_collection(
    DocumentRecord,
    collection_name="documents"
)

# Creer la collection (si elle n'existe pas)
# SK 1.39+: ensure_collection_exists() au lieu de create_collection_if_not_exists()
await collection.ensure_collection_exists()

print("Collection 'documents' creee")
print(f"Schema: id (key), content (data), title (data), embedding (vector 1536d)")

Collection 'documents' creee
Schema: id (key), content (data), title (data), embedding (vector 1536d)


In [4]:
# Documents d'exemple
documents = [
    {
        "id": "doc1",
        "title": "Introduction a Semantic Kernel",
        "content": "Semantic Kernel est un SDK open-source de Microsoft pour integrer des LLMs dans vos applications."
    },
    {
        "id": "doc2",
        "title": "Plugins SK",
        "content": "Les plugins dans Semantic Kernel sont des collections de fonctions que le kernel peut invoquer."
    },
    {
        "id": "doc3",
        "title": "Agents SK",
        "content": "L'Agent Framework permet de creer des agents autonomes qui utilisent des plugins et collaborent entre eux."
    },
    {
        "id": "doc4",
        "title": "RAG avec SK",
        "content": "RAG (Retrieval-Augmented Generation) combine la recherche vectorielle avec la generation de texte par LLM."
    }
]

# Generer les embeddings
contents = [doc["content"] for doc in documents]
embeddings = await embedding_service.generate_embeddings(contents)

# Creer les records
records = []
for doc, emb in zip(documents, embeddings):
    record = DocumentRecord(
        id=doc["id"],
        title=doc["title"],
        content=doc["content"],
        embedding=list(emb)
    )
    records.append(record)

# Inserer dans la collection (SK 1.39+: upsert prend une liste)
keys = await collection.upsert(records)
print(f"Documents inseres: {keys}")

Documents inseres: ['doc1', 'doc2', 'doc3', 'doc4']


In [5]:
# Recherche vectorielle
query = "Comment creer des agents avec Semantic Kernel ?"

# Generer l'embedding de la requete
query_embedding = (await embedding_service.generate_embeddings([query]))[0]

# Rechercher les documents similaires (SK 1.39+: search au lieu de vectorized_search)
results = await collection.search(
    vector=list(query_embedding),
    vector_property_name="embedding",
    top=3,
    include_vectors=False
)

print(f"Query: {query}")
print("\nResultats:")
print("-" * 60)

# SK 1.39+: results.results est un async generator
async for result in results.results:
    print(f"Score: {result.score:.4f}")
    print(f"Title: {result.record.title}")
    print(f"Content: {result.record.content}")
    print("-" * 60)

Query: Comment creer des agents avec Semantic Kernel ?

Resultats:
------------------------------------------------------------
Score: 0.3709
Title: Plugins SK
Content: Les plugins dans Semantic Kernel sont des collections de fonctions que le kernel peut invoquer.
------------------------------------------------------------
Score: 0.4582
Title: Agents SK
Content: L'Agent Framework permet de creer des agents autonomes qui utilisent des plugins et collaborent entre eux.
------------------------------------------------------------
Score: 0.4796
Title: Introduction a Semantic Kernel
Content: Semantic Kernel est un SDK open-source de Microsoft pour integrer des LLMs dans vos applications.
------------------------------------------------------------


### Interpretation : Recherche Vectorielle

La recherche vectorielle retourne les documents les plus proches semantiquement :

| Parametre | Description | Valeur typique |
|-----------|-------------|----------------|
| **top** | Nombre de resultats | 3-10 |
| **score** | Similarite (0-1 pour cosinus) | > 0.7 = bon match |
| **distance_function** | Mesure de distance | cosine, dotproduct, euclidean |

**Points cles** :
- Le document sur les "Agents SK" a le meilleur score car semantiquement lie a la question
- Le score indique la pertinence (plus haut = plus pertinent)
- Les resultats sont tries par score decroissant

## 5. Qdrant (Production)

Qdrant est un vector store production-ready. Nous avons une instance disponible.

In [6]:
from semantic_kernel.connectors.qdrant import QdrantStore
from qdrant_client import QdrantClient

# Configuration Qdrant (depuis .env ou valeurs fournies)
QDRANT_URL = os.getenv("QDRANT_URL", "https://qdrant.myia.io")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY", "4f89edd5-90f7-4ee0-ac25-9185e9835c44")

# Connexion au client Qdrant
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)

# Verification de la connexion
try:
    collections = qdrant_client.get_collections()
    print(f"Connexion Qdrant reussie !")
    print(f"Collections existantes: {[c.name for c in collections.collections]}")
except Exception as e:
    print(f"Erreur de connexion: {e}")
    print("Continuez avec InMemoryStore pour les exemples suivants.")

Erreur de connexion: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
Continuez avec InMemoryStore pour les exemples suivants.


In [7]:
# Creation du store Qdrant via SK
try:
    qdrant_store = QdrantStore(
        url=QDRANT_URL,
        api_key=QDRANT_API_KEY
    )
    
    # Collection Qdrant (SK 1.39+)
    qdrant_collection = qdrant_store.get_collection(
        DocumentRecord,
        collection_name="sk_demo"
    )
    
    await qdrant_collection.ensure_collection_exists()
    
    # Inserer les memes documents
    keys = await qdrant_collection.upsert(records)
    print(f"Documents inseres dans Qdrant: {keys}")
    
    # Recherche
    qdrant_results = await qdrant_collection.search(
        vector=list(query_embedding),
        vector_property_name="embedding",
        top=3
    )
    
    print(f"\nRecherche Qdrant pour: '{query}'")
    async for result in qdrant_results.results:
        print(f"  {result.score:.4f} - {result.record.title}")
        
except Exception as e:
    print(f"Qdrant non disponible: {e}")
    print("Les exemples utilisent InMemoryStore.")

Qdrant non disponible: All connection attempts failed
Les exemples utilisent InMemoryStore.


### Interpretation : InMemory vs Qdrant

| Caracteristique | InMemoryVectorStore | Qdrant |
|-----------------|---------------------|--------|
| **Persistance** | Non (RAM seulement) | Oui (disque/cloud) |
| **Scalabilite** | ~10K documents | Millions de documents |
| **Performance** | Rapide (petits datasets) | Optimise (HNSW index) |
| **Infrastructure** | Aucune | Serveur/Cloud |
| **Usage** | Dev/Test | Production |

**Qdrant specifiques** :
- Index HNSW pour recherche rapide
- Filtres sur metadonnees
- Sharding pour scalabilite horizontale
- API REST et gRPC

## 6. RAG Pattern Complet

Assemblons tout pour un pipeline RAG fonctionnel.

In [8]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.contents import ChatHistory

# Ajouter le service de chat
kernel.add_service(OpenAIChatCompletion(service_id="chat"))

async def rag_query(question: str, collection, embedding_service, kernel, top_k: int = 3):
    """Pipeline RAG complet."""
    
    # 1. Generer l'embedding de la question
    query_embedding = (await embedding_service.generate_embeddings([question]))[0]
    
    # 2. Rechercher les documents pertinents (SK 1.39+)
    results = await collection.search(
        vector=list(query_embedding),
        vector_property_name="embedding",
        top=top_k,
        include_vectors=False
    )
    
    # 3. Construire le contexte (SK 1.39+: async iteration)
    context_parts = []
    async for result in results.results:
        context_parts.append(f"- {result.record.title}: {result.record.content}")
    
    context = "\n".join(context_parts)
    
    # 4. Construire le prompt augmente
    augmented_prompt = f"""Tu es un assistant qui repond en utilisant uniquement le contexte fourni.
    
CONTEXTE:
{context}

QUESTION: {question}

REPONSE (basee uniquement sur le contexte):"""
    
    # 5. Appeler le LLM (SK 1.39+: settings requis)
    chat_service = kernel.get_service(service_id="chat")
    history = ChatHistory()
    history.add_user_message(augmented_prompt)
    
    settings = OpenAIChatPromptExecutionSettings()
    response = await chat_service.get_chat_message_contents(
        chat_history=history,
        settings=settings
    )
    
    return {
        "question": question,
        "context": context,
        "answer": str(response[0])
    }

# Test du pipeline RAG
result = await rag_query(
    question="Comment les agents SK peuvent-ils utiliser des plugins ?",
    collection=collection,
    embedding_service=embedding_service,
    kernel=kernel
)

print("=" * 60)
print(f"Question: {result['question']}")
print("=" * 60)
print(f"\nContexte utilise:\n{result['context']}")
print("=" * 60)
print(f"\nReponse:\n{result['answer']}")

Question: Comment les agents SK peuvent-ils utiliser des plugins ?

Contexte utilise:
- Agents SK: L'Agent Framework permet de creer des agents autonomes qui utilisent des plugins et collaborent entre eux.
- Plugins SK: Les plugins dans Semantic Kernel sont des collections de fonctions que le kernel peut invoquer.
- Introduction a Semantic Kernel: Semantic Kernel est un SDK open-source de Microsoft pour integrer des LLMs dans vos applications.

Reponse:
Les agents SK utilisent des plugins en invoquant les collections de fonctions fournies par ces plugins. Les plugins sont intégrés dans le Semantic Kernel, ce qui permet aux agents autonomes de collaborer entre eux en s'appuyant sur ces fonctions.


### Interpretation : Pipeline RAG

Le pipeline RAG suit ces etapes :

```
1. EMBEDDING          2. SEARCH           3. AUGMENT         4. GENERATE
┌─────────────┐     ┌─────────────┐     ┌─────────────┐    ┌─────────────┐
│  Question   │────>│  Vector     │────>│  Prompt +   │───>│    LLM      │
│  -> Vector  │     │  Search     │     │  Context    │    │  Response   │
└─────────────┘     └─────────────┘     └─────────────┘    └─────────────┘
```

**Avantages du RAG** :

| Avantage | Description |
|----------|-------------|
| **Donnees privees** | Le LLM peut acceder a vos documents |
| **Actualite** | Pas besoin de re-entrainer le modele |
| **Precision** | Reponses basees sur des sources |
| **Tracabilite** | On sait d'ou vient l'information |
| **Cout** | Moins cher que le fine-tuning |

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **VectorStore** | Abstraction de base | `InMemoryVectorStore()`, `QdrantStore()` |
| **Collection** | Groupe de records | `store.get_collection(name, type)` |
| **Record** | Document + embedding | `@vectorstoremodel @dataclass` |
| **Embedding** | Vecteur semantique | `OpenAITextEmbedding.generate_embeddings()` |
| **Search** | Recherche similitude | `collection.vectorized_search(vector, options)` |
| **RAG** | Retrieval-Augmented Generation | Contexte + LLM |

## Points cles a retenir

1. **InMemory pour dev, Qdrant pour prod** - Meme API, backend different
2. **Les embeddings capturent le sens** - Pas juste les mots-cles
3. **RAG = Search + Generate** - Contexte pertinent pour le LLM
4. **Le chunking est crucial** - Decouper les longs documents
5. **Les metadonnees enrichissent** - Filtres et contexte additionnel

## Exercices suggeres

1. **RAG sur PDF** : Ingerer un PDF et poser des questions
2. **Filtres** : Ajouter des filtres sur les metadonnees (date, auteur)
3. **Evaluation** : Mesurer la qualite des reponses RAG

## Pour aller plus loin

| Notebook | Contenu |
|----------|--------|
| [06-ProcessFramework](06-SemanticKernel-ProcessFramework.ipynb) | Workflows orchestres |
| [07-MultiModal](07-SemanticKernel-MultiModal.ipynb) | Images et audio |

---

**Navigation** : [<< 04-Filters](04-SemanticKernel-Filters-Observability.ipynb) | [Index](README.md) | [06-ProcessFramework >>](06-SemanticKernel-ProcessFramework.ipynb)